## BLERSSI Pipeline Deployment in UCS

### Import libraries

In [ ]:
import kfp
import os
from kubernetes import client
import calendar
import time

### Component files Declarations

In [ ]:
notebook_environment = "local" # or 'in-cluster'

if notebook_environment == "in-cluster":
    path='cisco-kubeflow-starter-pack/apps/networking/ble-localization/onprem/pipelines/'
    !git clone https://github.com/CiscoAI/cisco-kubeflow-starter-pack
else:
    path="apps/networking/ble-localization/onprem/pipelines/"

component_root_train= path+'components/v2/tf-model-train/'
component_root_serve = path+'components/v2/tf-serving-deploy-service/'
component_root_webui = path+'components/v2/tf-webui-deploy-service/'
component_root_tensorboard= path+'components/v2/tf-tensorboard-deploy-service/'

#### Components Description

tf_train_model_op &nbsp;&nbsp;&nbsp; - &nbsp; Load dataset from nfs-volume, train BLERSSI model  and save model in nfs-volume. <br> 
tf_serve_model_op &nbsp; - &nbsp; Load BLERSSI model from nfs-volume and serve the model using tf-serving for later prediction. <br> 
tf_webui_op &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - &nbsp; Web application client UI loads Unlabeled data, does preprocessing, send API request to serving and display Predicted Location. <br> 
tf_tensor_op &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  - &nbsp; Tensorboard UI which provides the visualization metrics such as loss and accuracy, model graph (ops and layers) and histograms of weights, biases, or other tensors for BLERSSI Model <br> 

### Load components from respective .YAML config files

In [ ]:
from kubernetes import config

if notebook_environment == "in-cluster":
    config.load_incluster_config()

tf_train_model_op = kfp.components.load_component_from_file(os.path.join(component_root_train, 'component.yaml')) 
tf_serve_model_op = kfp.components.load_component_from_file(os.path.join(component_root_serve, 'component.yaml'))
tf_webui_op = kfp.components.load_component_from_file(os.path.join(component_root_webui, 'component.yaml'))
tf_tensor_op = kfp.components.load_component_from_file(os.path.join(component_root_tensorboard, 'component.yaml'))

### Define Volume and Volume Mounts

In [ ]:
nfs_pvc = client.V1PersistentVolumeClaimVolumeSource(claim_name='nfs-wf-volume')
nfs_volume = client.V1Volume(name='nfs-wf-volume', persistent_volume_claim=nfs_pvc)
nfs_volume_mount = client.V1VolumeMount(mount_path='/mnt/', name='nfs')

### Define pipeline function

In [ ]:
from datetime import datetime

# Creating timestamp
timestamp = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')

def blerssi_pipeline():

    #Defining Task for Model training
    tf_train_model_task = tf_train_model_op()
    #tf_train_model_task.add_volume(nfs_volume)
    tf_train_model_task.add_volume_mount(nfs_volume_mount)
    
    #Defining Task to perform serving    
    tf_serve_model_task = tf_serve_model_op(timestamp=timestamp)
    #tf_serve_model_task.add_volume(nfs_volume)
    tf_serve_model_task.add_volume_mount(nfs_volume_mount)
    tf_serve_model_task.after(tf_train_model_task)

    #Defining task for tensorboard creation    
    tf_tensorboard_task = tf_tensor_op(timestamp=timestamp)
    #tf_tensorboard_task.add_volume(nfs_volume)
    tf_tensorboard_task.add_volume_mount(nfs_volume_mount)
    tf_tensorboard_task.after(tf_train_model_task)
    
    
    #Defining task for webUI creation
    tf_webui_task = tf_webui_op(image_path="docker.io/premkarthi/blerssi-webui:v4",
                                timestamp=timestamp)
    #tf_webui_task.add_volume(nfs_volume)
    tf_webui_task.add_volume_mount(nfs_volume_mount)
    tf_webui_task.after(tf_serve_model_task)

In [ ]:
if notebook_environment == "in-cluster":
    kfp_client = kfp.Client()
    kfp_client.create_run_from_pipeline_func(blerssi_pipeline, arguments={})
else:
    try:
        pipeline_package_name = "blerssi.yaml"
        kfp.compiler.Compiler().compile(blerssi_pipeline, pipeline_package_name)
    except RuntimeError:
        print("Validation failed but you can still run the workflow")